<a href="https://colab.research.google.com/github/vuduclyunitn/learning_python/blob/master/K%C4%A9_thu%E1%BA%ADt_ghi_nh%E1%BB%9B_trong_Python_L%C3%A0m_th%E1%BA%BF_n%C3%A0o_%C4%91%E1%BB%83_cache_c%C3%A1c_k%E1%BA%BFt_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dịch và bổ sung từ https://dbader.org/blog/python-memoization

Trong bài này, ta sẽ tìm hiểu cách làm cho chương trình chạy nhanh hơn với một kĩ thuật gọi là ghi nhớ (*memoization*)

**Kĩ thuật ghi nhớ là một kiểu caching chuyên dụng như là một kĩ thuật tối ưu**

Cache được dùng để lưu các kết quả của một hành đồng và sau đó được sử dụng bởi các hành động sau đó. Ví dụ, trình duyệt web của bạn sẽ sử dụng một cache để nạp bài viết này nhanh hơn khi bạn đọc lại bài hướng dẫn này.

Vì vậy khi tôi nói về sự ghi nhớ và Python nghĩa là tôi đang nói về việc ghi nhớ hay caching kết quả của một hàm dựa trên đầu vào của nó. 

Sự ghi nhớ cho phép ta tối ưu một hàm Python thông qua việc ghi lại kết quả của hàm này dựa trên các tham số bạn cung cấp cho nó. Mỗi khi bạn ghi nhớ một hàm, bạn sẽ chỉ tính toán kết quả của một hàm một lần cho một tập các tham số mà bạn cung cấp cho nó. Mọi lời gọi sau lời gọi đầu tiên sẽ được lấy về nhanh chống từ cache. 

Trong bài viết này, ta sẽ tìm hiểu cách vận dụng và khi nào thì vận dụng kĩ thuật đơn giản nhưng mạnh mẽ này, và bạn có thể sử dụng nó để tối ưu các chương trình của bạn và làm cho nó nhanh hơn trong một số trường hợp.

## Tại sao bạn nên sử dụng ghi nhớ và khi nào thì sử dụng ghi nhớ trong các chương trình của bạn.

Câu trở lời là những đoạn mã đắt

Khi bạn đang phân tích code, bạn chú ý đến nó chạy hết bao lâu và chiếm bao nhiêu bộ nhớ. Nếu đoạn mã bạn đang phân tích mất nhiều thời gian để chạy và tiêu tốn nhiều bộ nhớ, nó được gọi là code đắt (expensive code).

Nó đắt là bởi vì nó tốn rất nhiều tài nguyên, không gian và thời gian để chạy. Khi bạn chạy mã đắt, nó tốn rất nhiều tài nguyên và chiếm các tài nguyên từ các chương trình khác trong máy của bạn.

Nếu bạn muốn tăng tốc các phần code đắt này, sử dụng kĩ thuật ghi nhớ sẽ giúp bạn giải quyết được nhiều điều. Ta cùng tìm hiểu về kĩ thuật ghi nhớ trước khi triển khai nó.

Tất cả các ví dụ tôi sử dụng trong bài này được viết bằng Python 3, nhưng nó cũng có thể áp dụng cho Python 2.

## Giải thích giải thuật ghi nhớ

Giải thích ghi nhớ được diễn tả như sau:


1.   Thiết lập một cấu trúc dữ liệu cache để lưu kết quả các hàm
2.   Mỗi lần hàm được gọi, ta thực hiện những điều sau
      

      *   Trả về kết quả được ghi nhớ, nếu trong cache tồn tại.
      *    Gọi hàm để tính toán kết quả bị sót, và sau đó cập nhật lại cache trước khi trả quả cho mã gọi.

Với đủ lượng cache việc làm trên đảm bảo rằng các kết quả của một hàm với một tập các tham số sẽ chỉ được tính toán một lần.

Miễn là chúng ta có một kết quả được lưu lại chúng ta sẽ không cần phải chạy lại hàm được ghi nhớ này cho với tập đầu vào giống như vậy nữa. Thay vào đó, ta chỉ cần lấy kết quả đã được ghi nhớ và trả về ngay lập tức.




## Ta cùng viết một decorator ghi nhớ từ đầu

Ta sẽ triển khai giải thuật nhớ phía trên như là một ```decorator``` vì đây là một cách thuận tiện để triển khai một hàm chung hay là các hàm bao (wrappers) trong Python:

Một decorator là một hàm có đầu vào là một hàm khác và trả về một hàm khác. 

Dùng decorator cho phép ta triển khai giải thuật nhớ theo cách tổng quát và có thể tái sử dụng. Nếu bạn thấy nó còn mơ hồ, dễ lẫn lộn hãy nhìn vào các đoạn code dưới đây.

In [0]:
def memoize(func):
  cache = dict()
  
  def memoized_func(*args):
    if args in cache:
      return cache[args]
    result = func(*args)
    cache[args] = result
    return result
  
  return memoized_func

Decorator phía trên lấy vào một hàm và trả về một phiên bản bao (wrapped version) của cũng hàm đó, phiên bản bao này thực hiện này thực hiện việc ghi nhớ (```memoized_func```)

Ở đây ta sử dụng một từ điển (dictionary) như là một cache. Trong Python, khi ta muốn tìm kiếm một thứ gì từ điển. Do đó sử dụng ```dict``` cho phép ta lưu kết quả của hàm của hàm trong cache là tốt nhất.

Bất cứ khi nào hàm được decorate bị gọi, ta kiểm tra xem các tham số đã có sẵn trong cache hay không. Nếu chúng tồn tại, trả về kết quả được ghi nhớ. Thay vì tính toán lại kết quả, ta chỉ cần trả về kết quả từ cache. 

Nếu kết quả không tồn tại trong cache, bạn phải cập nhật lại để ta có thể tiết kiệm thời gian trong tương lai. Như vậy, đầu tiên ta tính toán kết quả còn thiếu, lưu nó vào trong cache, và trả kết quả về cho code gọi.


Hãy cùng kiểm tra decorator ghi nhớ của chúng ta với hàm chuỗi số Fibonacci. Đầu tiên, ta sẽ định nghĩa một hàm tính toán số Fibonacci thứ ```n```

In [0]:
def fibonacci(n):
  if n == 0:
    return 0
  elif n == 1:
    return 1
  return fibonacci(n-1) + fibonacci(n-2)

Hàm ```fibonacci``` phía trên là một ví dụ của một hàm tính toán đắt (expensive computation). Tính toán số Fibonacci thứ ```n``` có độ phức tạp thời gian ```O(2^n)``` - để hoàn thành nó ta mất thời gian theo cấp số mũ. Điều đó làm cho nó là một hàm đắt.

Tiếp theo, ta sẽ tạo một chuẩn để đo lường độ đắt tính toán của hàm. Sử dụng module có sẵn ```timeit```` để tính toán thời gian thực thi theo giây của một câu lệnh Python.

Dưới đây là code chúng ta sử dụng để đo lường thời gian thực thi của hàm ```fibonacci```

In [3]:
import timeit
timeit.timeit('fibonacci(35)', globals=globals(), number=1)

5.992539394999994

Như các bạn thấy, trên máy của tôi để thực hiện hàm này ta mất gần 6 giây. Rõ ràng đây là một hành động chậm và đắt.

Nào hãy xem sử dụng tính năng caching đem lại hiệu quả thế nào

In [8]:
memoized_fibonacci = memoize(fibonacci)
timeit.timeit('memoized_fibonacci(35)', globals=globals(), number=1)

5.979804820999789

Ta tiết kiệm được một chút thời gian khi sử dụng decorator ghi nhớ.

Tuy nhiên đó mới chỉ là lần chạy đầu tiên với một cache lạnh (cold) - là ta cache đó trống rỗng lúc đầu và không giúp ta cải thiện tốc độ gọi hàm gì nhiều. Nhưng hãy nhìn vào lần chạy thứ 2 hai này. 

In [9]:
timeit.timeit('memoized_fibonacci(35)', globals=globals(), number=1)

2.9860000267944997e-06

Chú ý vào chuỗi kí  tự ```e-06``` ở cuối, ta chỉ mất khoangr 2 micro giây để hoàn thành hàm ```memoized_fibonacci``` ở lần chạy thứ 2. Một sự tăng đáng kể.

Thay vì tính toán số Fibonacci thứ 35 ta chỉ cần lấy ra kết quả đã được ghi lại và trả nó về ngay lập tức, đó là lý do tại sao kết quả của ta nhanh hơn rất nhiều.

## Mổ xẻ Cache 

Để hiểu sâu hơn cách kĩ thuật ghi nhớ hoạt động, tôi muốn cho bạn xem nội dụng của cache được sử dụng trong ví dụ trước.

Để xem nội dung của cache bên trong hàm ```memoized_fibonacci``` ta sử dụng thuộc tính ```__closure``` của nó. 

In [11]:
memoized_fibonacci.__closure__[0].cell_contents

{(35,): 9227465}

Từ điển cache là biến cục bộ đầu tiên và được lưu trong cell số 0. Tôi không khuyến khích bạn sử dụng kĩ thuật này trong code chạy sản phẩm nhưng đây là một mẹo để debugging.

Như bạn có thể thấy, từ điển cache ánh xạ các tham số là các tuples và kết quả của hàm. Ví dụ ở đây ta thấy kết quả cho số fibonacci thứ 35 là 9227465.

In [12]:
fibonacci(35)

9227465

Ta cùng làm một thực nghiệm nhỏ để biểu diễn cách hoạt động của cache. Tôi sẽ gọi ```memoized_fibonacci``` một vài lần để lấp giá trị vào cache và ta cùng mổ xẻ kết quả của nó một lần nữa

In [13]:
memoized_fibonacci(1)


1

In [14]:
memoized_fibonacci(2)

1

In [15]:
memoized_fibonacci(3)

2

In [16]:
memoized_fibonacci(4)

3

In [17]:
memoized_fibonacci(5)

5

In [18]:
memoized_fibonacci.__closure__[0].cell_contents

{(1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (35,): 9227465}

Như bạn có thể thấy, từ điển cache hiện tại chứa kết quả cho một vài inputs khác nhau cho hàm ```memoized_fibonacci```. Nó sẽ giúp ta lấy ra kết quả nhanh hơn từ cache thay vì tính toán lại chúng từ đầu.

**Cảnh báo**: trong ví dụ này kích thước cache của ta không được giới hạn, nghĩa là cache có thể phát triển tự nhiên như nó muốn. Đây không phải là một cách làm tốt bởi vì có thể dẫn tới các lỗi về hết bộ nhớ trong chương trình của bạn.

Với bất cứ kiểu ghi nhớ nào, ta cần đặt giới hạn lượng dữ liệu nó có thể giữ trong cache tại cùng một thời điểm. Ta có thể làm việc đó thông qua việc thiết lập các giới hạn cứng về kích thước cache hay định nghĩa một chính sách hết hạn để trục xuất các phần tử khỏi cache tại một số thời điểm.

Nhớ rằng hàm ```memoize``` ta viết ở trên chỉ nhằm mục đích giới thiệu kĩ thuật ghi nhớ. Trong phần sau bạn sẽ thấy được cách triển khai trong sản phẩm.

## Sử dụng ```functools.lru_cache``` hỗ trợ cho việc ghi nhớ

Bạn đã thấy được cách triển khai một hàm ghi nhớ. Trong phần này tôi sẽ chỉ cho bạn một cách tiện hơn để đạt được kết quả giống như vậy sử dụng ```functools.lru_cache```.

Decorator có tên ```functools.lru_cache``` là một triển khai ghi nhớ lấy ra từ thư viện chuẩn. Một khi bạn nhận ra thời điểm bạn sử dụng ```lru_cache```, bạn có thể tăng tốc chương chình của mình với chỉ vài dòng code.

Hãy xem lại ví dụ về số Fibonacci ta đã làm trước đó. Nhưng bây giờ ta sẽ sử dụng decorator ```functools.lru_cache```:

In [0]:
import functools
@functools.lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

Chú ý rằng tham số ```maxsize``` là để giới hạn số lượng các phần tử được lưu vào cache tại cùng một thời điểm.

Mọt lần nữa tôi sử dụng module ```timeit``` để chạy các benchmark đơn giản vì vậy ta có thể thấy được sự cải tiến

In [20]:
timeit.timeit('fibonacci(35)', globals=globals(), number=1)

3.2387999908678466e-05

In [21]:
timeit.timeit('fibonacci(35)', globals=globals(), number=1)

3.37999972543912e-06

Bạn có thể thắc mắc tại sao ngay tại lần đầu mà hiệu năng đã cao như vậy (thời gian thực thi thấp). Cache không bị lạnh ngay ở lần đầu hay sau?

Sự khác biệt ở đây là ở chỗ ta áp dụng decorator ```@lru_cache``` tại thời điểm hàm này được định nghĩa. Có nghĩa là các lời gọi đệ qui tới hàm ```fibonacci()``` cũng được truy vấn trong cache tại thời điểm.

Qua việc decorate hàm ```fibonacci()``` với decorator ```@lru_cache``` tôi đã chuyển lời giải trước đó sang lời giải ứng dụng lập trình động (dynamic programming), nơi mỗi vấn đề con được giải chỉ một lần thông qua việc lưu các lời giải của các vấn đề con và truy vấn chúng từ cache trong lần tiếp theo.

## Tại sao bạn nên lựa chọn ```functools.lru_cache````

Nói chung, sử dụng ```functools.lru_cache``` toàn diện hơn rất nhiều so với hàm ghi nhớ tính thế (ad hoc memoize function)

Ví dụ, ```functools.lru_cache```` cung cấp một tính năng hữu dụng cho phép bạn lấy ra các thống kê về ghi nhớ với phương thức  ```cache_info```:

In [22]:
fibonacci.cache_info()

CacheInfo(hits=34, misses=36, maxsize=128, currsize=36)

Như bạn có thể thấy trên, ```lru_cache()``` nhớ các lời gọi đệ quy tới ```fibonacci()```, cache trong trường hợp này trả về 34 kết quả được ghi nhớ, đó là lý do tại sao chương trình của ta nhanh hơn.

```functools.lru_cache``` cho phép bạn giới hạn số lượng các kết quả được ghi nhớ thông qua tham số ```maxsize```.  Nếu bạn thiết lập ```maxsize=None``` cache sẽ trở nên không có giới hạn đó là điều tôi thương không khuyến khích.

Có một biến kiểu boolean tên là ```typed```, khi bạn cho nó giá trị True nó sẽ tách bị các tham số hàm dựa trên kiểu của tham số. Ví dụ, ```fibonacci(35)``` sẽ khác với ```fibonacci(35.0)```.

Một tính năng hữu dụng khác là khả năng thiết lập lại (reset) cache tại bất cứ thời điểm nào với phương thức ```cache_clear```

In [0]:
fibonacci.cache_clear()

In [24]:
fibonacci.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

Tống kết lại một điều là bạn nên sử dụng ```lru_cache()``` vì nó có sẵn, toàn diện và đã được kiểm tra tốt hơn so với hàm nhớ của bạn.

## Những điều cần thận trọng với ghi nhớ - Cái cần được ghi nhớ

Lý tưởng nhất là khi bạn muốn nhớ các hàm xác định (deterministic). Ví dụ:

In [0]:
def deterministic_adder(x, y):
  return x + y

Ở đây hàm ```deterministic_adder()``` là một hàm xác định bởi vì nó sẽ luôn luôn trở về kết quả giống nhau với các cặp tham số giống nhau. Ví dụ, nếu bạn gửi vào nó hai số 2 và 3, nó sẽ luôn trả về số 5.


So sánh với một hàm không xác định (nondeterministic)

In [0]:
from datetime import datetime

def nondeterministic_adder(x, y):
  # Check to see if today is Monday 
  if datetime.now().weekday() == 0:
    return x + y + x
  return x + y

Hàm trên là hàm không xác định bởi vì kết quả của nó cho một input xác định sẽ phụ thuộc vào ngày trong tuần: Nếu bạn chạy hàm này vào ngày thứ hai, cahce sẽ trả về một dữ liệu khác với các ngày khác. 

Nói chung tôi thấy rằng bất cứ hàm nào cập nhật một record hay trả về thông tin thay đổi thương xuyên thì không nên để nhớ. 

## Tổng kết lại
Trong bài viết này bạn biết được cách ghi nhớ giúp bạn tối ưu một hàm thông qua việc ghi nhớ kết quả của hàm dựa trên các tham số cung cấp cho nó.

Mỗi khi bạn nhớ một nhàm, nó sẽ chỉ tính toán chỉ một lần kết quả của nó cho một tập các tham số bạn truyền vào. Các lời gọi sau với cùng một tập tham số kết quả sẽ được trả về từ cache.

Bạn đã thấy cách viết một decorator của riêng bạn, và tại sao ta nên sử dụng ```lru_cache()```.


*   Ghi nhớ là một kĩ thuật tối ưu hóa phần mềm, nó lưu và trả về kết quả của một lời gọi hàm dựa trên các tham số của nó.
*   Nếu chương trình của bạn thoả mãn các điều kiện đó, kĩ thuật nhớ có thể cải tiến chương trình của bạn
* Bạn có import một hàm ghi nhớ toàn diện, ```lru_cache()``` từ thư viện chuẩn Python ```functools```

